 The challenge is to predict Workers Compensation claims using highly realistic synthetic data.

 The evaluation method is Root Mean Squared Error (RMSE)

In [53]:
import gensim
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.cluster import KMeans
from sklearn.metrics import (
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
    accuracy_score
)
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor, plot_tree

In [5]:
plt.style.use('dark_background')
# Disable scientific notation
pd.set_option('display.float_format', lambda x: '%.9f' % x)
# pd.reset_option('display.float_format')

In [ ]:
df = pd.read_csv("train.csv", parse_dates=["DateReported", "DateTimeOfAccident"])

df.info()

### Preprocessing

In [7]:
df = df.drop_duplicates()

In [ ]:
# replace nan values with unknown 
df["MaritalStatus"] = df["MaritalStatus"].fillna('Unknown')
df["MaritalStatus"].unique()

In [9]:
df[["Gender", "MaritalStatus", "PartTimeFullTime"]] = df[
    ["Gender", "MaritalStatus", "PartTimeFullTime"]
].astype("category")

### Feature Engineering

In [10]:
df["DateOfAccident"] = df["DateTimeOfAccident"].dt.date
df["TimeOfAccident"] = df["DateTimeOfAccident"].dt.time

col_index = df.columns.get_loc("DateTimeOfAccident")

# make sure it's int to suppress error
if not isinstance(col_index, int):
    raise ValueError("`datetime_col_index` must be an integer.")

df.insert(col_index + 1, "DateOfAccident", df.pop("DateOfAccident"))
df.insert(col_index + 2, "TimeOfAccident", df.pop("TimeOfAccident"))

df["DateOfAccident"] = pd.to_datetime(df["DateOfAccident"], format="ISO8601", utc=True)
# df["TimeOfAccident"] = pd.to_datetime(df["TimeOfAccident"], format="%H:%M:%S", utc=True)

In [11]:
# Convert dates to format a model can understand
df["AccidentDay"] = df["DateTimeOfAccident"].dt.day
df["AccidentMonth"] = df["DateTimeOfAccident"].dt.month
df["AccidentYear"] = df["DateTimeOfAccident"].dt.year
df["AccidentWeekday"] = df["DateTimeOfAccident"].dt.weekday
df["ReportedDay"] = df["DateReported"].dt.day
df["ReportedMonth"] = df["DateReported"].dt.month
df["ReportedYear"] = df["DateReported"].dt.year
df["ReportedWeekday"] = df["DateReported"].dt.weekday
df["DaysToReportAccident"] = (df.DateReported - df.DateOfAccident).dt.days

In [ ]:
df.skew(numeric_only=True)

In [15]:
df[["LogInitialIncurredCalimsCost", "LogUltimateIncurredClaimCost"]] = np.log1p(
    df[["InitialIncurredCalimsCost", "UltimateIncurredClaimCost"]]
)

In [ ]:
df[["LogInitialIncurredCalimsCost", "LogUltimateIncurredClaimCost"]].head()

In [ ]:
df[["InitialIncurredCalimsCost", "UltimateIncurredClaimCost"]].head()

In [ ]:
df.skew(numeric_only=True)

In [17]:
stops = stopwords.words("english")


def text_clean(claim):
    # Converting to Lower Case
    claim = claim.lower()
    # Getting List Of Words
    claim = claim.split()
    # Removing Stop Words(Words which do not add any information like =is,are,I etc)
    claim = [word for word in claim if word not in stops]
    # Stemming the word(words like playing ,played are replaced with play)
    porter_stemmer = PorterStemmer()
    stem_claim = [porter_stemmer.stem(word) for word in claim]

    # Lemmatizing the words (replacing words with their base form)
    lemmatizer = WordNetLemmatizer()
    lem_claim = [lemmatizer.lemmatize(word) for word in claim]
    return lem_claim, stem_claim

In [18]:
df["ClaimDescriptionClean"] = df["ClaimDescription"].apply(
    lambda x: " ".join(text_clean(x)[0])
)

In [19]:
tokenized_descriptions = [claim.split() for claim in df["ClaimDescriptionClean"]]

# Train the Word2Vec model
model = Word2Vec(
    sentences=tokenized_descriptions,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4,
)


def create_embedding(claim):
    words = claim.split()
    embeddings = [model.wv[word] for word in words if word in model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.vector_size)

In [20]:
df["ClaimDescriptionEmbedding"] = df["ClaimDescriptionClean"].apply(create_embedding)

In [ ]:
all_embeddings = np.array([embedding for embedding in df["ClaimDescriptionEmbedding"]])
all_embeddings.shape

In [ ]:
# Perform K-means clustering
kmeans = KMeans(n_clusters=5, random_state=42)
cluster_labels = kmeans.fit_predict(all_embeddings)

# Add cluster labels to the dataframe
df["ClusterLabel"] = cluster_labels

# View some claims from each cluster
for cluster in range(5):
    print(f"\nCluster {cluster} claims:")
    print(df[df["ClusterLabel"] == cluster]["ClaimDescriptionClean"].head(3))

In [ ]:
df_encoded = pd.get_dummies(
    df[
        [
            # "ClaimNumber",
            # "DateTimeOfAccident",
            # "DateOfAccident",
            # "TimeOfAccident",
            # "DateReported",
            "Age",
            "Gender",
            "MaritalStatus",
            "DependentChildren",
            "DependentsOther",
            "WeeklyWages",
            "PartTimeFullTime",
            "HoursWorkedPerWeek",
            "DaysWorkedPerWeek",
            # "ClaimDescription",
            "InitialIncurredCalimsCost",
            "UltimateIncurredClaimCost",
            # "LogInitialIncurredCalimsCost",
            # "LogUltimateIncurredClaimCost",
            "AccidentDay",
            "AccidentMonth",
            "AccidentYear",
            "AccidentWeekday",
            "ReportedDay",
            "ReportedMonth",
            "ReportedYear",
            "ReportedWeekday",
            #"ClaimDescriptionClean",
            #"ClaimDescriptionEmbedding",
            "ClusterLabel",
            
        ]
    ],
)
df_encoded.shape

In [ ]:
df_scaled = pd.get_dummies(
    df[
        [
            # "ClaimNumber",
            # "DateTimeOfAccident",
            # "DateOfAccident",
            # "TimeOfAccident",
            # "DateReported",
            "Age",
            "Gender",
            "MaritalStatus",
            "DependentChildren",
            "DependentsOther",
            "WeeklyWages",
            "PartTimeFullTime",
            "HoursWorkedPerWeek",
            "DaysWorkedPerWeek",
            # "ClaimDescription",
            # "InitialIncurredCalimsCost",
            #"UltimateIncurredClaimCost",
            "LogInitialIncurredCalimsCost",
            "LogUltimateIncurredClaimCost",
            "AccidentDay",
            "AccidentMonth",
            "AccidentYear",
            "AccidentWeekday",
            "ReportedDay",
            "ReportedMonth",
            "ReportedYear",
            "ReportedWeekday",
            #"ClaimDescriptionClean",
            #"ClaimDescriptionEmbedding",
            "ClusterLabel",
            
        ]
    ],
)
df_scaled.shape

### Train model

In [39]:
X = df_encoded.drop(
    columns=["UltimateIncurredClaimCost"]#, 
)
y = df_encoded["UltimateIncurredClaimCost"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [40]:
Xs = df_scaled.drop(
    columns=["LogUltimateIncurredClaimCost"]#, 
)
ys = df_scaled["LogUltimateIncurredClaimCost"]

X_trains, X_tests, y_trains, y_tests = train_test_split(
    Xs, ys, test_size=0.2, random_state=42
)

In [41]:
params = {
    "objective": "reg:squarederror",  # Regression objective
    "n_estimators": 100,  # Number of trees (boosting rounds)
    "learning_rate": 0.01,  # Learning rate
    "max_depth": 3,  # Maximum depth of each tree
    "min_child_weight": 3,  # Minimum sum of instance weight (hessian) in a child
    "subsample": 0.8,  # Proportion of training data used for each tree
    "colsample_bytree": 0.8,  # Fraction of features used for each tree
    "gamma": 0.1,  # Minimum loss reduction to make a further partition on a leaf node
    "reg_alpha": 0,  # L1 regularization term
    "reg_lambda": 0,  # L2 regularization term
    "seed": 42,  # Random seed for reproducibility
}


In [42]:
model = XGBRegressor(**params)
model.fit(X_train, y_train);
models = XGBRegressor(**params)
models.fit(X_trains, y_trains);

### Evaluate model

In [ ]:
y_pred = model.predict(X_test)
y_preds = models.predict(X_tests)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)
rmses = np.sqrt(mean_squared_error(y_tests, y_preds))
print("RMSES:", rmses)
# print("Unscaled RMSE:", np.exp(rmse))

In [ ]:
print(len(y_pred), len(X_test))

In [ ]:
mape = mean_absolute_percentage_error(y_pred,y_test)
mapes = mean_absolute_percentage_error(y_preds,y_tests)
print(mape,mapes)

In [ ]:
# Assuming you have RMSE values before and after scaling
rmse_before = 26454.83716219436
rmse_after = 0.8844200359201072

# Calculate percentage reduction
percentage_reduction = (rmse_before - rmse_after) / rmse_before * 100

print("RMSE before scaling:", rmse_before)
print("RMSE after scaling:", rmse_after)
print("Percentage reduction:", percentage_reduction)

In [ ]:
np.sqrt(np.exp(mean_squared_error(y_test, y_pred)))

In [ ]:
rmse_before = 26454.83716219436
rmse_after_log = 0.8844200359201072

# Back-transform the RMSE
rmse_after_original_scale = np.exp(rmse_after_log) - 1

print(f"RMSE before log transformation: {rmse_before}")
print(f"RMSE after log transformation (back-transformed): {rmse_after_original_scale}")

In [ ]:
# Calculate percentage errors
def percentage_error(y_true, y_pred):
    return (y_pred - y_true) / y_true * 100

# Original model
pe_original = percentage_error(y_test, y_pred)

# Log-transformed model
pe_log = percentage_error(np.exp(y_tests), np.exp(y_preds))  # Assuming y_tests and y_preds are log-transformed

# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot for the original model
sns.histplot(pe_original, kde=True, ax=ax1)
ax1.set_xlabel("Percentage Error (%)")
ax1.set_ylabel("Frequency")
ax1.set_title("Distribution of Percentage Errors: Original Model")

# Plot for the log-transformed model
sns.histplot(pe_log, kde=True, ax=ax2)
ax2.set_xlabel("Percentage Error (%)")
ax2.set_ylabel("Frequency")
ax2.set_title("Distribution of Percentage Errors: Log-transformed Model")

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

# Print summary statistics
print("Original Model - Percentage Error Statistics:")
print(f"Mean: {pe_original.mean():.2f}%")
print(f"Median: {np.median(pe_original):.2f}%")
print(f"Standard Deviation: {pe_original.std():.2f}%")

print("\nLog-transformed Model - Percentage Error Statistics:")
print(f"Mean: {pe_log.mean():.2f}%")
print(f"Median: {np.median(pe_log):.2f}%")
print(f"Standard Deviation: {pe_log.std():.2f}%")

In [ ]:
# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot for the original model
sns.scatterplot(x=y_test, y=y_pred, ax=ax1)
ax1.set_xlabel("Actual Values")
ax1.set_ylabel("Predicted Values")
ax1.set_title("Original Model: Actual vs. Predicted")
ax1.ticklabel_format(style='plain')

# Plot for the log-transformed model
sns.scatterplot(x=y_tests, y=y_preds, ax=ax2)
ax2.set_xlabel("Actual Values (Log-transformed)")
ax2.set_ylabel("Predicted Values (Log-transformed)")
ax2.set_title("Log-transformed Model: Actual vs. Predicted")

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def percentage_error(y_true, y_pred):
    return (y_pred - y_true) / y_true * 100

# Calculate percentage errors
pe_original = percentage_error(y_test, y_pred)
pe_log = percentage_error(np.exp(y_tests), np.exp(y_preds))  # Assuming y_tests and y_preds are log-transformed

# Create a figure with six subplots
fig, axs = plt.subplots(3, 2, figsize=(20, 24))

# Scatter plot for original model
axs[0, 0].scatter(y_test, y_pred)
axs[0, 0].set_xlabel("Actual Values")
axs[0, 0].set_ylabel("Predicted Values")
axs[0, 0].set_title("Original Model: Actual vs. Predicted")
axs[0, 0].ticklabel_format(style='plain')

# Scatter plot for log-transformed model
axs[0, 1].scatter(y_tests, y_preds)
axs[0, 1].set_xlabel("Actual Values (Log-transformed)")
axs[0, 1].set_ylabel("Predicted Values (Log-transformed)")
axs[0, 1].set_title("Log-transformed Model: Actual vs. Predicted")

# Percentage error distribution for original model
sns.histplot(pe_original, kde=True, ax=axs[1, 0])
axs[1, 0].set_xlabel("Percentage Error (%)")
axs[1, 0].set_ylabel("Frequency")
axs[1, 0].set_title("Distribution of Percentage Errors: Original Model")

# Percentage error distribution for log-transformed model
sns.histplot(pe_log, kde=True, ax=axs[1, 1])
axs[1, 1].set_xlabel("Percentage Error (%)")
axs[1, 1].set_ylabel("Frequency")
axs[1, 1].set_title("Distribution of Percentage Errors: Log-transformed Model")

# Feature importance for original model
feature_importance_orig = model.feature_importances_
axs[2, 0].bar(X_train.columns, feature_importance_orig)
axs[2, 0].set_xlabel("Features")
axs[2, 0].set_ylabel("Importance")
axs[2, 0].set_title("Feature Importance: Original Model")
axs[2, 0].tick_params(axis='x', rotation=90)

# Feature importance for log-transformed model (assuming you have this)
# If you don't have this, you can remove this subplot or duplicate the original model's feature importance
feature_importance_log = models.feature_importances_  # Assuming you have this
axs[2, 1].bar(X_trains.columns, feature_importance_log)
axs[2, 1].set_xlabel("Features")
axs[2, 1].set_ylabel("Importance")
axs[2, 1].set_title("Feature Importance: Log-transformed Model")
axs[2, 1].tick_params(axis='x', rotation=90)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

# Print summary statistics
print("Original Model - Percentage Error Statistics:")
print(f"Mean: {pe_original.mean():.2f}%")
print(f"Median: {np.median(pe_original):.2f}%")
print(f"Standard Deviation: {pe_original.std():.2f}%")

print("\nLog-transformed Model - Percentage Error Statistics:")
print(f"Mean: {pe_log.mean():.2f}%")
print(f"Median: {np.median(pe_log):.2f}%")
print(f"Standard Deviation: {pe_log.std():.2f}%")

In [ ]:
# Extract feature importance
feature_importance = model.feature_importances_

# Plot the feature importance scores
plt.bar(X_train.columns, feature_importance)
plt.xlabel("Features")
plt.ylabel("Importance")
plt.title("Feature Importance")
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Create a DataFrame with y_pred and y_test as columns
df_pred = pd.DataFrame({"y_pred": y_pred, "y_test": y_test})

# Print the DataFrame
df_pred["diff"] = df_pred["y_test"] - df_pred["y_pred"]
df_pred.head(20).apply(lambda x: x.apply('{0:.5f}'.format))

In [ ]:
df_pred.describe().apply(lambda x: x.apply('{0:.5f}'.format))

In [ ]:
df_pred.plot(kind='box')
plt.title('Box Plot of Predicted Values')
plt.ylabel('Predicted Values')
plt.show()